In [1]:
#### REMEMBER TO GIT PULL!  ####
#### Add this block to all jupyter files ####

#Loading and importing necessary tools. 
%load_ext autoreload 
%autoreload 2
import sys, os
sys.path.append("scripts")
import load_params as LP

# Comment and uncomment these two lines for when you are working on this notebook. 
#param_file = "param_files/ryo_local.csv"
param_file = "param_files/binkley_local.csv" 

paramDict = LP.load_param_file(param_file)

In [2]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
from scipy import stats
import sys
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import sys
import codecs
 

from random import sample

import seaborn as sns
sns.set_style('whitegrid')

# guarantee unicode string
_u = lambda t: t.decode('UTF-8', 'replace') if isinstance(t, str) else t
_uu = lambda *tt: tuple(_u(t) for t in tt) 
# guarantee byte string in UTF8 encoding
_u8 = lambda t: t.encode('UTF-8', 'replace') if isinstance(t, unicode) else t
_uu8 = lambda *tt: tuple(_u8(t) for t in tt) 




In [3]:
#####This will be the iterative step

In [4]:
###Leave more blank space

In [3]:
# import custom functions
import deprecated_funcs as DF


In [6]:
# Potentially deprecated functions. 

DF.output_coldspot('coldspotlist.txt')
DF.extract_best_eqtl()

NameError: name 'paramDict' is not defined

In [9]:
#### This step outputs the cold spots and returns the list 

def ID_hotspots(outfilename): 
    fileIN = open(paramDict["gtexDir"] + 'coldspots.txt')
    hotspots = list()
    fileOUT = open(outfilename, "w")
    for i in fileIN: 
        i = i.rstrip().split('\t')

        
        Chr = i[0]
        for num in i[0]: #Get rid of "chr in the file"
            if num in "chr":
                Chr = Chr.replace(num, '')       
        start = int(i[1])
        end = int(i[2])
        middle = str(i[3])
        bound = str (i[4])
        hotspots.append([Chr, start, end, middle, bound])
        #hotspots.append(start)
        #hotspots.append(end)
        #hotspots.append(middle)
        fileOUT.write(Chr + "\t" + str(start) + "\t" + str(end) + "\t" + str(middle)  +  "\t" + str(bound) + "\n")
    fileIN.close()
    fileOUT.close()
    return hotspots
#ID_hotspots('coldspots2.txt')



In [14]:
#####OUtputs cold spot for given eqtl
def find_variant_coldspot(outfilename):
    fileIN = open( "besteQTLs.txt" )
    
    fileOUT = open(outfilename, "w")
    hspots = ID_hotspots('coldspots2.txt')



    for i in fileIN:
        
        i = i.rstrip().split('\t')
        gene = str(i[0])
        chrnum = str(i[1])
        snp = (i[3])
        position = float(i[2])
        pvalue = float(i[4])
        
        for  Chr, start, (end), middle, (bound) in hspots:
            if Chr  != chrnum :
                continue
            else: 
                if position < float(bound) and position > float(end): 
                    #distance = abs(position - middle)
                    fileOUT.write("\t".join([ str(Chr), str(start), str(end), str(middle), str(bound) , gene, snp, str(position), str(pvalue)] ) + "\n")
                else: 
                    continue
    fileIN.close()
    fileOUT.close()

find_variant_coldspot('eQTLcoldspot.txt')

#### This step can probably be truncated 
def output_selected_coldspots(outfilename):
    fileIN = open ('eQTLcoldspot.txt')
    fileOUT = open(outfilename, "w")
    selectedcoldspots = list()
    
    for i in fileIN:
        
        i = i.rstrip().split('\t')
        Chr = str(i[0])
        end = str(i[2])
        bound = str(i[4])
        
    

        fileOUT.write("\t".join([ str(Chr),  str(end),  str(bound)] ) + "\n")
        selectedcoldspots.append([ str(Chr),  str(end),  str(bound)])                    
    
    fileIN.close()
    fileOUT.close()
    return selectedcoldspots
output_selected_coldspots('selectedcoldspots.txt')




[['22', '40382500', '40525500']]

In [15]:
###THis outputs a file with GWAS variants in our cold spots of interest that colocalize
def find_variant_coldspot(outfilename):
    fileIN = open(paramDict["gtexDir"] +'/oncoarray_bcac_public_release_oct17.txt')
    fileIN.readline()
    fileOUT = open(outfilename, "w")


    cspots = output_selected_coldspots('selectedcoldspots.txt')

    for i in fileIN:
        
        i = i.rstrip().split('\t')
        gene = str(i[0])
        chrnum = str(i[2])
        #snp = (i[3])
        position = float(i[3])
        pvalue = float(i[9])
        if pvalue<10e-9:
            for  (Chr), (end), (bound) in cspots:
                if Chr  != chrnum:
                    continue
                else: 
                    if position < float(bound) and position > float(end): 
                    #distance = abs(position - middle)
                        fileOUT.write("\t".join([ str(Chr), str(end),  str(bound) , gene, str(position), str(pvalue)] ) + "\n")
                    else: 
                        continue
    fileIN.close()
    fileOUT.close()
find_variant_coldspot("gwasColdSpot.txt")



In [16]:
#####This step outputs the selected cold spot
def output_selected_coldspots(outfilename):
    fileIN = open ('selectedcoldspots.txt')
    fileOUT = open(outfilename, "w")
    selectedcoldspots2 = list()
    
    for i in fileIN:
        
        i = i.rstrip().split('\t')
        Chr = str(i[0])
        end = float(i[1])
        bound = float(i[2])
        selectedcoldspots2.append([ (Chr),  (end),  (bound)])              
    

        fileOUT.write("\t".join([ str(Chr),  str(end),  str(bound)] ) + "\n")
              
    
    fileIN.close()
    fileOUT.close()
    return selectedcoldspots2
output_selected_coldspots('preVCF.txt')



[['22', 40382500.0, 40525500.0]]

In [34]:
#####This step has many readlines, it outputs all variants in the cold spot

def filter_vcf(outfilename): 
    fileIN = open(paramDict["gtexDir"] +'/chr22_subset_gtex.vcf')
    
    selectedcoldspots2 = output_selected_coldspots('preVCF.txt')
    fileOUT = open(outfilename, "w")
    
    for i in fileIN:
        line= fileIN.readline()
        if line[0]=="#":
            continue

        i = i.rstrip().split('\t')
        
        chrnum = str(i[0])
        position = float(i[1])


        for  Chr, end, bound in selectedcoldspots2:
            if (Chr)  == chrnum:
                if position > end  and position < bound:  

                    fileOUT.write("\t".join([str(x) for x in i] ) + "\n")                    
                else: 
                    continue
      
    fileIN.close()
    fileOUT.close()
filter_vcf('filteredgenotype.txt')



In [10]:
######This step randomly picks an eQTL in the coldspot and the position

def select_rand_variant(outfilename, paramDict): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    rnd = list()
    for i in fileIN:

        i = i.rstrip().split('\t')
        rnd.append(i[2])

    rand = sample(rnd[0:],1)

    for i in rand:
        fileOUT.write("\t".join([str(x) for x in rand] ) +  "\n")

    fileOUT.close()
    fileIN.close()
    return rand
select_rand_variant(paramDict["gtexDir"] + "eQTLcausal.txt", paramDict)

def output_pos_rand_variant(outfilename, paramDict): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    sand = open(paramDict["gtexDir"] +  'eQTLcausal.txt')
    for i in sand: 
        i = i.rstrip().split('\t') 
        rand = str(i[0])

    for j in fileIN: 
        j = j.rstrip().split('\t')
        SNP = str(j[2])
        random = str(rand)

        if SNP == random: 

            fileOUT.write(str(j[1]) + "\n")
            break
    fileOUT.close()
    fileIN.close()
output_pos_rand_variant(paramDict["gtexDir"] + "eQTLcausalpos.txt", paramDict)    

In [11]:
#####This step randomly picks a GWAScausal in the coldspot

def select_rand_variant(outfilename): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    rnd = list()
    for i in fileIN:

        i = i.rstrip().split('\t')
        rnd.append(i[2])
    rand = sample(rnd[0:],1)

    for i in rand:
        fileOUT.write("\t".join([str(x) for x in rand] ) +  "\n")
    fileOUT.close()
    fileIN.close()
select_rand_variant(paramDict["gtexDir"] + "GWAScausal.txt")

def output_pos_rand_variant(outfilename, paramDict): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    sand = open(paramDict["gtexDir"] +  'GWAScausal.txt')
    for i in sand: 
        i = i.rstrip().split('\t') 
        rand = str(i[0])

    for j in fileIN: 
        j = j.rstrip().split('\t')
        SNP = str(j[2])
        random = str(rand)

        if SNP == random: 

            fileOUT.write(str(j[1]) + "\n")
            break
    fileOUT.close()
    fileIN.close()
output_pos_rand_variant(paramDict["gtexDir"] + "GWAScausalpos.txt", paramDict)    


In [12]:
import os

os.system("plink --vcf chr22_subset_gtex.vcf --show-tags eQTLcausal.txt --tag-r2 0.5")

512

In [13]:
import os

os.system("plink --vcf chr22_subset_gtex.vcf --show-tags GWAScausal.txt --tag-r2 0.5")

512

In [27]:
### This will be the GWAS to run the RTC score 
def select_rand_variant(outfilename): 
    fileIN = open(paramDict["gtexDir"] +  '/plink.tags')
    fileOUT = open(outfilename, "w")
    rnd = list()
    for i in fileIN:

        i = i.rstrip().split('\t')
        rnd.append(i[0])
    rand = sample(rnd[0:],1)

    for i in rand:
        fileOUT.write("\t".join([str(x) for x in rand] ) +  "\n")
    fileOUT.close()
    fileIN.close()
select_rand_variant(paramDict["gtexDir"] + "linkedGWAS.txt")

def output_pos_rand_variant(outfilename, paramDict): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    sand = open(paramDict["gtexDir"] +  'linkedGWAS.txt')
    posI = list()
    for i in sand: 
        i = i.rstrip().split('\t') 
        rand = str(i[0])

    for j in fileIN: 
        j = j.rstrip().split('\t')
        SNP = str(j[2])
        random = str(rand)

        if SNP == random: 

            fileOUT.write(str(j[1]) + "\n")
            posI.append(j[1])
            break
    return posI
    fileOUT.close()
    fileIN.close()
output_pos_rand_variant(paramDict["gtexDir"] + "linkedGWASpos.txt", paramDict)    



[]

In [31]:
###This will be the eQTL to run the RTC score 
def select_rand_variant(outfilename): 
    fileIN = open(paramDict["gtexDir"] +  '/plink.tags')
    fileOUT = open(outfilename, "w")
    rnd = list()
    for i in fileIN:

        i = i.rstrip().split('\t')
        rnd.append(i[0])
    rand = sample(rnd[0:],1)

    for i in rand:
        fileOUT.write("\t".join([str(x) for x in rand] ) +  "\n")
    fileOUT.close()
    fileIN.close()
select_rand_variant(paramDict["gtexDir"] + "linkedeQTL.txt")

def output_pos_rand_variant(outfilename, paramDict): 
    fileIN = open('filteredgenotype.txt')
    fileOUT = open(outfilename, "w")
    sand = open(paramDict["gtexDir"] +  'linkedeQTL.txt')
    posI = "BLAH" 
    for i in sand: 
        i = i.rstrip().split('\t') 
        rand = str(i[0])

    for j in fileIN: 
        j = j.rstrip().split('\t')
        SNP = str(j[2])
        random = str(rand)

        if SNP == random: 

            fileOUT.write(str(j[1]) + "\n")
            posI = j[1]
            break
    return posI
    fileOUT.close()
    fileIN.close()
output_pos_rand_variant(paramDict["gtexDir"] + "linkedeQTLpos.txt", paramDict)    

'40479570'

In [19]:
### RUn this for the real eQTL
import load_params as LP
import eqtl_funcs as EF

geneName = "ENSG00000230415.1"
chrNum = "22"
pos = "40051275"
tissue = "Breast_Mammary_Tissue"


slope, intercept,  residuals, indivVector = EF.get_eqtl_stats(chrNum, pos, geneName, tissue, paramDict)  

print(slope)
print(intercept)
print(residuals)



	Getting dosage vector for : 22 40051275 /Users/michaelbinkley/Desktop/GTEx/
	Getting header
	 calculate residuals
-0.0496456198632
0.0587440203162
[-0.77860703  1.39810775 -0.09874075 -0.34096017 -0.29382959  0.75299025
 -0.283329    1.26118757 -1.86148711  1.19085168  0.78254021  1.74399907
  0.02091565  1.48587787 -0.46175831 -0.47929349  0.79622181  0.16019533
  1.65929279  0.51891839 -1.53520175  1.56665057 -1.42468657 -2.15669228
  0.81739883 -2.66385743 -0.85640294 -0.14958395 -1.34207439  0.81005585
  1.11280207 -0.44542886 -1.20904848  0.83820614  1.90540742 -0.56341078
 -0.06869112  1.84663152  1.58412042  0.71622449  0.47687768  0.87369409
 -0.48391088 -0.21952679  0.1306119  -0.97651997  0.24081669  2.13849548
  1.42763539 -0.34691683  0.46038608  1.6205558  -1.7189053   0.69026735
  0.26115631 -1.39618657  1.45613539  0.60419472  1.2504277  -1.01040323
  0.90218134 -0.40111983 -1.04217204 -0.42893726  0.76900521 -1.56788195
  1.27698861 -1.00878889 -0.01894577  0.72757775 

In [21]:


def calculate_pp(slope_r, intercept, residuals, chrNum, pos, indivVector, paramDict):
    '''
    Calculates pseudophenotype for SNP at chrnum and pos, given slope,intersect, residuals. 
    IndivVector - ensures that the we are using the same individuals for this calculation as when the slope, intercept, and residuals were calculated.
    
    '''
    permutedResiduals = random.sample(list(residuals), len(residuals))
    genoVector, indivGenoVector = GF.get_dos_vector(chrNum, pos, paramDict["vcfDir"])
    filteredGenoVector, indivVector = GF.filter_and_sort_genotype_vector(genoVector, indivGenoVector, indivVector)
    pp = float(slope_r)*np.array(filteredGenoVector) + permutedResiduals + intercept   
    return pp



In [32]:
### run this for the simulated, use the position from the rand_linked functions
import eqtl_funcs as EF
import gtex_funcs as GF
import random
random.seed(0)

param_file = "param_files/binkley_local.csv" 

paramDict = LP.load_param_file(param_file)

geneName = "ENSG00000230415.1"
chrNum = "22"
pos = "40051275"
tissue = "Breast_Mammary_Tissue"


slope_r, intercept,  residuals, indivVector = EF.get_eqtl_stats(chrNum, pos, geneName, tissue, paramDict)  


chrNumLinked = "22"
### THIS is the position of the randomly selected
posLinked = output_pos_rand_variant(paramDict["gtexDir"] + "linkedeQTLpos.txt", paramDict)   

print("position", posLinked)

pp = calculate_pp(slope_r, intercept, residuals, chrNumLinked, posLinked, indivVector, paramDict)


	Getting dosage vector for : 22 40051275 /Users/michaelbinkley/Desktop/GTEx/
	Getting header
	 calculate residuals
position 40479570
	Getting dosage vector for : 22 40479570 /Users/michaelbinkley/Desktop/GTEx/
	Getting header


In [36]:


def output_gene_info(geneName, paramDict):
    
    fileIN = open(paramDict["tmpDir"] + '/chrname2.txt')
    selected = list()
    for i in fileIN: 
        i = i.rstrip().split('\t')
        
        if i[3]== geneName:
            selected = i
            break
            
    return selected

  
def output_header(indVector): 

            
            
    header = "\t".join(['#chr',  'start', 'end' , 'gene' , 'length' , 'strand'] + [str(x) for x in indVector] )
    return header

    fileIN.close()

def output_bed(indivVector, expVector, filepath, geneName, paramDict):
    selected = output_gene_info(geneName, paramDict)
    header = output_header(indivVector)
    with open(filepath, 'w') as outfile:
        outfile.write(header + "\n")
        geneLocation = selected
        outfile.write("\t".join([str(x) for x in geneLocation]) + "\t" + "\t".join([str(x) for x in expVector]) + "\n")
print(geneName)        
output_bed(indivVector, pp, paramDict["tmpSmall"] + "/GTExExpfile.bed", geneName, paramDict)






ENSG00000230415.1


In [38]:
### Seems to be an error when indexing with the header, but worked after removing the header, 
##not sure we even need to index the PP file

import os

os.system("bgzip /users/michaelbinkley/desktop/RTCstuffs/tmpsmall/GTExExpfile.bed")
os.system("tabix -p bed /users/michaelbinkley/desktop/RTCstuffs/tmpsmall/GTExExpfile.bed.gz")


256

In [40]:
#### Calc RTC score 

import os


os.system("QTLtools rtc --vcf /users/michaelbinkley/desktop/GTEx/chr22_subset_gtex.vcf.gz' --bed /users/michaelbinkley/desktop/RTCstuffs/tmpsmall/GTExExpfile.bed.gz --hotspot hotspots_b37_hg19.bed --gwas-cis GWAS.b37.txt permutations_all.significant.txt --normal --out rtc_results.txt")



512

In [ ]:
#### For a given eQTL (the best eqtl for a given gene) run simulation with the most sign gwas hit in coldspot
#### TRIAL 1 - use the best overall eQTL within any coldspot for a given gene